In [9]:
import pandas as pd
import time
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
import pymongo

In [2]:
# Configure ChromeDriver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\RebekahDSK\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [3]:
# # Insert into Mongo DB

def scrape_all():

    # Populate variables from the functions
    news_title, news_p = mars_news()
    featured_img_url = featured_image()
    mars_facts_html = mars_facts()

    # Assemble the document to insert into the database
    nasa_document = {
        'news_title': news_title,
        'news_paragraph': news_p,
        'featured_img_url': featured_img_url,
        'mars_facts_html': mars_facts_html
    }

    # consider closing browser here
    browser.quit()

    return nasa_document

# NASA Mars News

In [4]:
def mars_news():
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)

    html = browser.html
    news_soup = BeautifulSoup(html, 'html.parser')

    article_container = news_soup.find('ul', class_='item_list')

    headline_date = article_container.find('div', class_='list_date').text
    news_title = article_container.find('div', class_='content_title').find('a').text
    news_p = article_container.find('div', class_='article_teaser_body').text

    return news_title, news_p

# JPL Mars Space Images - Featured Image

In [5]:
def featured_image():
    base_url = 'https://www.jpl.nasa.gov'

    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)

    html = browser.html
    img_soup = BeautifulSoup(html, 'html.parser')

    # Method 1: parsing through the style attribute in the article tag
    try:
        img_elem = img_soup.find('article', class_='carousel_item')['style']
        img_rel_url = img_elem.replace("background-image: url('", '')
        img_rel_url = img_rel_url.replace("');", '')
        #print(img_rel_url)
    except Exception as e:
        print(e)

    # Method 2: clicking the FULL TEXT button and pulling the image
    try:
        full_image_elem = browser.find_by_id('full_image')[0]
        full_image_elem.click()

        html = browser.html
        img_soup = BeautifulSoup(html, 'html.parser')

        img_rel_url = img_soup.find('img', class_='fancybox-image')['src']
        #print(img_rel_url)
    except Exception as e:
        print(e)

    featured_image_url  = f'{base_url}{img_rel_url}'
    print(featured_image_url)
    
    return featured_image_url

# Mars Facts

In [6]:
def mars_facts():
    url = 'https://space-facts.com/mars/'
    browser.visit(url)

    mars_facts_df = pd.read_html(url)
    mars_facts_df = mars_facts_df[0]
    mars_facts_df.columns = ['Description', 'Mars']
    mars_facts_df

    mars_facts_html = mars_facts_df.to_html(classes='table table-striped', index=False, border=0)
    
    return mars_facts_html

# Mars Hemispheres

In [7]:
# def mars_hemis():
#     url4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
#     browser.visit(url4)
#     html = browser.html
#     soup = BeautifulSoup(html, 'html.parser')
#     mars_hemis=[]
    
#     for i in range (4):
#         time.sleep(5)
#         images = browser.find_by_tag('h3')
#         images[i].click()
#         html = browser.html
#         soup = BeautifulSoup(html, 'html.parser')
#         partial = soup.find("img", class_="wide-image")["src"]
#         img_title = soup.find("h2",class_="title").text
#         img_url = 'https://astrogeology.usgs.gov'+ partial
#         dictionary={"title":img_title,"img_url":img_url}
#         mars_hemis.append(dictionary)
#         browser.back()
    
#     mars_hemis['mars_hemis'] = mars_hemis
    # mars_hemis.to_html(classes='table table-striped', index=False, border=0)
    # return mars_hemis_html

In [10]:
# Instantiate Flask app
app = Flask(__name__)

# Connect to MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Connect to mars_app database
db = client.mars_app

# Connect to mars collection
mars_coll = db.mars

@app.route('/')
def index():

    mars_data = mars_coll.find_one()

    return(render_template('index.html', mars_data=mars_data))

@app.route('/scrape')
def scrape():

    # this is the py script with all of the scraping functions
    import scrape_mars

    # Gather document to insert
    nasa_document = scrape_mars.scrape_all()

    # Insert into the mars collection
    # mars_coll.insert_one(nasa_document)

    # Upsert into the mars collection (preferred to avoid duplicates)
    mars_coll.update_one({}, {'$set': nasa_document}, upsert=True)

    return redirect('/')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\RebekahDSK\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
